In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[0] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[0] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[0]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #訓練資料(要用幾秒訓練)
    dataX = dataX1
    dataY = dataY1

    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型(只訓練分類器)
    train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_3052\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 135.89 seconds
----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/60 Train_Loss: 3.4209091663360596 Train_acc: 5.175000%
Train Epoch: 2/60 Train_Loss: 2.7701685428619385 Train_acc: 11.337501%
Train Epoch: 3/60 Train_Loss: 3.0308871269226074 Train_acc: 17.525002%
Train Epoch: 4/60 Train_Loss: 2.7096877098083496 Train_acc: 20.775002%
Train Epoch: 5/60 Train_Loss: 2.466895818710327 Train_acc: 23.887501%
Train Epoch: 6/60 Train_Loss: 2.4257116317749023 Train_acc: 25.625002%
Train Epoch: 7/60 Train_Loss: 2.5851664543151855 Train_acc: 28.337502%
Train Epoch: 8/60 Train_Loss: 2.2943532466888428 Train_acc: 30.237501%
Train Epoch: 9/60 Train_Loss: 2.582059621810913 Train_ac

Train Epoch: 45/60 Train_Loss: 1.1803299188613892 Train_acc: 69.212502%
Train Epoch: 46/60 Train_Loss: 1.0681698322296143 Train_acc: 72.075005%
Train Epoch: 47/60 Train_Loss: 0.8052064180374146 Train_acc: 72.812500%
Train Epoch: 48/60 Train_Loss: 1.1979987621307373 Train_acc: 73.162506%
Train Epoch: 49/60 Train_Loss: 0.9647107720375061 Train_acc: 73.275002%
Train Epoch: 50/60 Train_Loss: 1.008109211921692 Train_acc: 73.412506%
Train Epoch: 51/60 Train_Loss: 0.7230499982833862 Train_acc: 73.487503%
Train Epoch: 52/60 Train_Loss: 0.7003423571586609 Train_acc: 73.825005%
Train Epoch: 53/60 Train_Loss: 0.7829810976982117 Train_acc: 73.925003%
Train Epoch: 54/60 Train_Loss: 0.769197940826416 Train_acc: 73.750000%
Train Epoch: 55/60 Train_Loss: 0.7794058918952942 Train_acc: 74.137505%
Train Epoch: 56/60 Train_Loss: 0.987734854221344 Train_acc: 74.800003%
Train Epoch: 57/60 Train_Loss: 1.0950244665145874 Train_acc: 74.237503%
Train Epoch: 58/60 Train_Loss: 1.0274909734725952 Train_acc: 74.837

Train Epoch: 29/60 Train_Loss: 1.764559030532837 Train_acc: 55.512501%
Train Epoch: 30/60 Train_Loss: 1.470848798751831 Train_acc: 53.975002%
Train Epoch: 31/60 Train_Loss: 1.406349539756775 Train_acc: 58.650002%
Train Epoch: 32/60 Train_Loss: 1.2058864831924438 Train_acc: 59.750004%
Train Epoch: 33/60 Train_Loss: 1.177111029624939 Train_acc: 60.500004%
Train Epoch: 34/60 Train_Loss: 1.3882967233657837 Train_acc: 61.200005%
Train Epoch: 35/60 Train_Loss: 1.288130283355713 Train_acc: 61.387505%
Train Epoch: 36/60 Train_Loss: 1.0998119115829468 Train_acc: 61.087502%
Train Epoch: 37/60 Train_Loss: 1.3104274272918701 Train_acc: 61.162502%
Train Epoch: 38/60 Train_Loss: 1.4009190797805786 Train_acc: 61.675003%
Train Epoch: 39/60 Train_Loss: 1.2212841510772705 Train_acc: 61.612503%
Train Epoch: 40/60 Train_Loss: 1.428719162940979 Train_acc: 62.562504%
Train Epoch: 41/60 Train_Loss: 1.1860843896865845 Train_acc: 62.937504%
Train Epoch: 42/60 Train_Loss: 0.9858658313751221 Train_acc: 62.950005

In [8]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()